In [2]:
from typing import Annotated
from dotenv import load_dotenv
load_dotenv()

from typing_extensions import TypedDict
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [3]:
from langchain.chat_models import init_chat_model

In [4]:
llm = init_chat_model("openai:gpt-3.5-turbo")

class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: State) -> State:
    return {"messages": [llm.invoke(state["messages"])]}

builder = StateGraph(State)
builder.add_node("chatbot_node", chatbot)

builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

graph = builder.compile()

In [5]:
message = {"role": "user", "content": "Who walked on the moon for the first time? Print only the name"}
# message = {"role": "user", "content": "What is the latest price of MSFT stock?"}
response = graph.invoke({"messages":[message]})

response["messages"]

[HumanMessage(content='Who walked on the moon for the first time? Print only the name', additional_kwargs={}, response_metadata={}, id='cb304022-b288-4415-968d-f541ce0aa3a6'),
 AIMessage(content='Neil Armstrong', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 21, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Bt1uqoDQBOU8x7RjCkOL6s3CSd6mw', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--2ddfa757-6400-41c3-8192-6d794b780d20-0', usage_metadata={'input_tokens': 21, 'output_tokens': 2, 'total_tokens': 23, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]

In [6]:
state = None
while True:
    print("Enter 'quit' or 'exit' to end the chat")
    in_message = input("You: ")
    if in_message.lower() in {"quit","exit"}:
        break
    if state is None:
        print("Initializing state...")
        state: State = {
            "messages": [{"role": "user", "content": in_message}]
        }
    else:
        state["messages"].append({"role": "user", "content": in_message})

    state = graph.invoke(state)
    print("Bot:", state["messages"][-1].content)

Enter 'quit' or 'exit' to end the chat
Initializing state...
Bot: The first person to walk on the moon was Neil Armstrong on July 20, 1969 during the Apollo 11 mission.
Enter 'quit' or 'exit' to end the chat
Bot: The second person to walk on the moon was Buzz Aldrin, who followed Neil Armstrong shortly after on July 20, 1969 during the Apollo 11 mission.
Enter 'quit' or 'exit' to end the chat
Bot: The third person to walk on the moon was Charles "Pete" Conrad, who was part of the Apollo 12 mission and walked on the moon on November 14, 1969.
Enter 'quit' or 'exit' to end the chat
Bot: Based on your previous questions, it seems like you are asking about the astronauts who walked on the moon during the Apollo missions. Let me know if you have any other questions or need more information.
Enter 'quit' or 'exit' to end the chat
